I'm using the final version of the Data--Frame throughout. To get it you can simply run Labeled_Data.py which lives in the data folder. 

In [ ]:
import pandas as pd

# Go up one level, then into Data/
df = pd.read_csv('../Data/amazon_merged.csv',nrows=1000)

# Check it's loaded
print(df.shape)
df.head()


(1000, 21)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,match,incident_indices
0,"['Toys & Games', 'Puzzles', 'Jigsaw Puzzles']",NaN,"[""Three Dr. Suess' Puzzles: Green Eggs and Ham...",NaN,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,[],NaN,Dr. Seuss,"['Three giant floor puzzles', 'Includes: Dr. S...","['>#2,230,717 in Toys & Games (See Top 100 in ...",...,Toys & Games,NaN,NaN,NaN,0000191639,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN,0,[]
1,[],NaN,['<b>Prepare to be Afraid!</b><br /><br />The ...,NaN,Pathfinder: Book of Beasts - Legendary Foes,[],NaN,Pathfinder Roleplaying Jon Brazer Productions,[],"['>#2,294,535 in Toys & Games (See Top 100 in ...",...,Toys & Games,NaN,NaN,.a-box-inner{background-color:#fff}#alohaBuyBo...,0004950763,[],[],NaN,0,[]
2,[],NaN,[],NaN,Nursery Rhymes Felt Book,[],NaN,Betty Lukens,[],"['>#2,871,983 in Toys & Games (See Top 100 in ...",...,Toys & Games,NaN,NaN,.a-box-inner{background-color:#fff}#alohaBuyBo...,0005069491,[],[],NaN,0,[]
3,"['Toys & Games', 'Games', 'Card Games']",NaN,['Blitz Vintage Card Game'],NaN,Dutch Blitz Card Game,['B00WIIPQEC'],NaN,Dutch Blitz Games Co,[],"['>#376,337 in Toys & Games (See Top 100 in To...",...,Toys & Games,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$24.95,0004983289,[],[],NaN,0,[]
4,[],NaN,"['Marvel SuperHeroes themed ""magic ink"" colori...",NaN,Magic Pen Painting (Marvel Super Heroes),"['B009SB7TI8', 'B009SB8Z1I']",NaN,Lee Publications,[],"['>#934,954 in Toys & Games (See Top 100 in To...",...,Toys & Games,NaN,NaN,$4.92,0006466222,[],[],NaN,0,[]


In [3]:
import pandas as pd

# Assume df is already defined and has 'asin' and 'matches'
# Step 1: Count how many unique match values per asin
nunique_per_asin = df.groupby('asin')['match'].nunique()

# Step 2: Get list of asins with more than one unique match value
conflicting_asins = nunique_per_asin[nunique_per_asin > 1].index.tolist()

# Step 3: Filter your dataframe to only those conflicting asins
conflicting_rows = df[df['asin'].isin(conflicting_asins)].sort_values('asin')

# Step 4: Display result
print(f"⚠️ Found {len(conflicting_asins)} ASINs with both 0 and 1 in 'matches'.")
conflicting_rows.head()


⚠️ Found 0 ASINs with both 0 and 1 in 'matches'.


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,match,incident_indices


The asins have not been dropped but they are consistently labeled. 

We now investigate the category feature, in particular we want to figure out if we can replace our dataset with a smaller one by removing some unnecessary categories. 

In [4]:
# Step 1: Group by category and filter to those where all matches == 0
groups = df.groupby('category')
only_zeros = groups.filter(lambda g: (g['match'] == 0).all())

# Step 2: Count number of listings per such category
counts = only_zeros['category'].value_counts()

# Step 3: Build result list
result = sorted(list(counts.items()), key=lambda x: x[1])  # ascending by count

# Step 4: Stats
num_categories = len(result)
total_listings = sum(counts)

# Step 5: Display
print(f"📊 Number of categories with only match == 0: {num_categories}")
print(f"📦 Total Amazon listings in these categories: {total_listings}")
print("\n🔝 Top 10 categories with only 0's (by listing count):")
for category, count in result[-10:][::-1]:  # top 10 descending
    print(f"{category}: {count}")


📊 Number of categories with only match == 0: 104
📦 Total Amazon listings in these categories: 1000

🔝 Top 10 categories with only 0's (by listing count):
['Toys & Games', 'Games', 'Board Games']: 208
[]: 132
['Toys & Games', 'Games', 'Card Games']: 98
['Toys & Games', 'Arts & Crafts', 'Drawing & Painting Supplies', 'Drawing & Sketch Pads']: 39
['Toys & Games', "Kids' Electronics", 'Electronic Learning Toys']: 37
['Toys & Games', 'Puzzles', 'Jigsaw Puzzles']: 37
['Toys & Games', 'Games']: 36
['Toys & Games', 'Arts & Crafts', 'Stickers']: 27
['Toys & Games', 'Puzzles']: 22
['Toys & Games', 'Action Figures & Statues', 'Action Figures']: 18


In [5]:
# Step 1: Group by category and filter those with at least one match == 1
groups = df.groupby('category')
has_one_match = groups.filter(lambda g: (g['match'] == 1).any())

# Step 2: Count number of listings per such category
counts_match_1 = has_one_match['category'].value_counts()

# Step 3: Build result list sorted descending by count
result_match_1 = sorted(list(counts_match_1.items()), key=lambda x: x[1], reverse=True)

# Step 4: Stats
num_categories_match_1 = len(result_match_1)
total_listings_match_1 = sum(counts_match_1)

# Step 5: Display
print(f"✅ Number of categories with at least one match == 1: {num_categories_match_1}")
print(f"📦 Total Amazon listings in these categories: {total_listings_match_1}")
print("\n🔝 Top 10 categories with at least one match == 1 (by listing count):")
for category, count in result_match_1[:10]:
    print(f"{category}: {count}")


✅ Number of categories with at least one match == 1: 0
📦 Total Amazon listings in these categories: 0

🔝 Top 10 categories with at least one match == 1 (by listing count):


In [6]:
num_all_categories = df['category'].nunique()
print(f"📊 Total number of unique categories: {num_all_categories}")


📊 Total number of unique categories: 104


In [7]:
# Step 1: Extract categories that had at least one match == 1
categories_with_match_1 = [category for category, count in result_match_1]

# Step 2: Filter the original DataFrame to keep only those rows
smaller_df = df[df['category'].isin(categories_with_match_1)].copy()

# Step 3: Confirm
print(f"✅ smaller_df created with {len(smaller_df)} rows across {smaller_df['category'].nunique()} categories.")


✅ smaller_df created with 0 rows across 0 categories.


In [8]:
# Count how many listings per category
category_counts = smaller_df['category'].value_counts()

# Find max and min
max_listings = category_counts.max()
min_listings = category_counts.min()

# Find which categories those are
category_with_max = category_counts.idxmax()
category_with_min = category_counts.idxmin()

# Display
print(f"🔝 Max listings in a single category: {max_listings} (Category: '{category_with_max}')")
print(f"🔻 Min listings in a single category: {min_listings} (Category: '{category_with_min}')")


ValueError: attempt to get argmax of an empty sequence

In [ ]:
# Group by category
grouped = smaller_df.groupby('category')

# 1️⃣ Count categories with only one listing
category_sizes = grouped.size()
num_single_listing_categories = (category_sizes == 1).sum()
print(f"📦 Number of categories with only ONE listing: {num_single_listing_categories}")

# 2️⃣ Count categories where all listings have match == 1
only_ones = grouped['match'].apply(lambda x: (x == 1).all())
num_all_match_1 = only_ones.sum()
print(f"✅ Number of categories where all listings have match == 1: {num_all_match_1}")


In [ ]:
import matplotlib.pyplot as plt

# Step 1: Count listings per category
category_counts = smaller_df['category'].value_counts().head(20)

# Step 2: Plot
plt.figure(figsize=(10, 8))
category_counts.sort_values().plot(kind='barh')  # horizontal bar chart
plt.title('Top 20 Categories by Number of Listings (with at least one match == 1)')
plt.xlabel('Number of Listings')
plt.ylabel('Category')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Step 1: Count listings per category (20 categories with the fewest listings)
category_counts = smaller_df['category'].value_counts().tail(20)
category_counts = category_counts.sort_values()  # Ensure smallest on top

# Step 2: Plot
plt.figure(figsize=(10, 8))
ax = category_counts.plot(kind='barh', color='skyblue')
plt.title('20 Categories with the Least Number of Listings\n(with at least one match == 1)')
plt.xlabel('Number of Listings')
plt.ylabel('Category')
plt.tight_layout()

# Step 3: Add value labels on the bars
for i, (value, name) in enumerate(zip(category_counts.values, category_counts.index)):
    ax.text(value + 0.5, i, str(value), va='center', fontsize=9)

plt.show()


I suggest we drop the only 0 categories and work with smaller_df. Below I consider the description category and I use NLP to extract some new features. Below you see "certification" feature. 

In [ ]:
from transformers import pipeline
from tqdm import tqdm

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Optional: checks if GPU is available


In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
text = "FDA certified and ISO approved non-toxic product."
labels = ["certified", "non-toxic", "hypoallergenic", "safe to use"]

result = classifier(text, labels, multi_label=True)
print(result)


In [ ]:
max_certified_score = max(result['scores'])  # Or result['scores'][result['labels'].index('certified')]
print(f"Max score for 'certified': {max_certified_score:.4f}")  

In [ ]:
certification_labels = [
    "FDA approved",
    "ISO certified",
    "CE marking",
    "third-party tested",
    "lab tested",
    "clinically tested",
    "hypoallergenic",
    "non-toxic",
    "PVC free",
    "BPA free",
    "phthalate free",
    "GMP certified",
    "UL certified",
    "organic certified"
]


In [ ]:
def get_all_certifications_with_score(text, labels=certification_labels, threshold=0.98):
    """
    Returns (labels_above_threshold, max_score) from zero-shot classification.
    If no label passes threshold or text is empty, returns (None, max_score).
    """
    if not text or not text.strip() or text.strip() == '[]':
        return (None, 0.0)

    result = classifier(text, labels, multi_label=True)
    high_conf_labels = [
        label for label, score in zip(result['labels'], result['scores']) if score > threshold
    ]
    max_score = max(result['scores']) if result['scores'] else 0.0

    return (high_conf_labels if high_conf_labels else None, max_score)


In [ ]:
text_1 = "FDA and CE certified product."
text_2 = "Gentle soap for everyday use."
text_3 = ""

print(get_all_certifications_with_score(text_1))  # ✅ should return some labels
print(get_all_certifications_with_score(text_2))  # ❌ should return None (scores < 0.95)
print(get_all_certifications_with_score(text_3))  # ❌ should return None (empty)


In [ ]:
sample_df = df.sample(n=300, random_state=42).copy()

from tqdm import tqdm
tqdm.pandas()

results = sample_df['description'].progress_apply(get_all_certifications_with_score)
sample_df['certification_labels'], sample_df['max_cert_score'] = zip(*results)



In [ ]:
certified_df = sample_df[sample_df['certification_labels'].notna()]


In [ ]:
# Show first 5 certified entries
certified_df[['description', 'certification_labels', 'max_cert_score']].head(10)


In [ ]:
# Total number of rows in the sample
total = len(sample_df)

# Count of rows where certification_labels is not None
with_certification = sample_df['certification_labels'].notna().sum()

# Percentage
percentage = (with_certification / total) * 100

print(f"{with_certification} out of {total} listings got at least one certification label.")
print(f"That's {percentage:.2f}% of the listings.")
